In [1]:
import numpy as np
f = np.array([123, 456, 789])
p = np.exp(f) / np.sum(np.exp(f))
print (p)

[ 0.  0. nan]


<ipython-input-1-0eff16e97f8a>:3: RuntimeWarning: overflow encountered in exp
  p = np.exp(f) / np.sum(np.exp(f))
<ipython-input-1-0eff16e97f8a>:3: RuntimeWarning: invalid value encountered in true_divide
  p = np.exp(f) / np.sum(np.exp(f))


In [15]:
#f2 = [24.5, 164, 0.18]
f1 = [3.2, 5.1, -1.7]
f1 -= np.max(f1) # f becomes [-666, -333, 0]
p = np.exp(f1) / np.sum(np.exp(f1)) # safe to do, gives the correct answer
l = -np.log(p)
print(p)
print(l)

[0.12998254 0.86904954 0.00096793]
[2.04035515 0.14035515 6.94035515]


In [2]:
import torch
k = [1, 2, 1, 1, 1]
x = torch.rand(5, 3)
p0 = torch.exp(x) / torch.sum(torch.exp(x))
pp = torch.exp(x[:, k]) / torch.sum(torch.exp(x))
print(p0,'\n')
print(pp)
loss = torch.sum(pp)
print(loss)

tensor([[0.0850, 0.0791, 0.0420],
        [0.0678, 0.0930, 0.0518],
        [0.0676, 0.0594, 0.0606],
        [0.0843, 0.0703, 0.0810],
        [0.0668, 0.0482, 0.0433]]) 

tensor([[0.0791, 0.0420, 0.0791, 0.0791, 0.0791],
        [0.0930, 0.0518, 0.0930, 0.0930, 0.0930],
        [0.0594, 0.0606, 0.0594, 0.0594, 0.0594],
        [0.0703, 0.0810, 0.0703, 0.0703, 0.0703],
        [0.0482, 0.0433, 0.0482, 0.0482, 0.0482]])
tensor(1.6786)


In [33]:
# vector-wise
import numpy as np
import torch
reg = 0.01
X = torch.rand(4, 3) # N * D = 4 * 3
W = torch.rand(3, 5) # D * C = 3 * 5
#y = torch.randint(4, (4,)) # N = 4
y = torch.randperm(4)
num_train = X.shape[0]
num_classes = W.shape[1]
print('X=', X, '\n')
print('W=', W, '\n')
print('y=', y, '\n')

for j in range(5):
    W[:, j] += X[0]
    print(W[:, j])

X= tensor([[0.5543, 0.8334, 0.9069],
        [0.3404, 0.5974, 0.1944],
        [0.9386, 0.9261, 0.2960],
        [0.8999, 0.1369, 0.8688]]) 

W= tensor([[0.0324, 0.7355, 0.1957, 0.3008, 0.3746],
        [0.2613, 0.0940, 0.3617, 0.0796, 0.6467],
        [0.7514, 0.1582, 0.6392, 0.9462, 0.5388]]) 

y= tensor([2, 0, 1, 3]) 

tensor([0.5867, 1.0947, 1.6583])
tensor([1.2899, 0.9274, 1.0651])
tensor([0.7501, 1.1951, 1.5461])
tensor([0.8552, 0.9130, 1.8531])
tensor([0.9289, 1.4801, 1.4457])


In [143]:
s = X.mm(W)
pp = s[:, y]
#print(score_0-torch.max(score_0))
print(score,'\n')
print(score[torch.arange(num_train),y],'\n')
#print(score[torch.arange(num_train), :])
print(torch.sum(score, axis=1))

tensor([[-0.9458, -0.4032, -0.2765, -0.6470, -0.4572],
        [-0.7259, -0.7193, -0.5570, -0.7627, -0.6967],
        [-0.6899, -0.3192, -0.0485, -0.4018, -0.2511],
        [-0.7217, -0.1981,  0.0000, -0.4174, -0.2181]]) 

tensor([-0.2765, -0.7193, -0.4018, -0.7217]) 

tensor([-2.7298, -3.4616, -1.7105, -1.5554])


In [53]:
# vector-wise
loss_vec = 0
dW_vec = torch.zeros_like(W)
score = X.mm(W) # N * C = 4 * 5
b = torch.arange(num_train)
score[b, :] -= torch.max(score[b, :])
#score_t = W.t().mm(X.t()) # C * N = 5 * 4

#print(torch.exp(score[b, :]), '\n', torch.sum(torch.exp(score), axis = 1), '\n')
p0 = torch.exp(score[b, :].t()) / torch.sum(torch.exp(score), axis = 1) # N * C = 4 * 5
p = torch.exp(score[b, y]) / torch.sum(torch.exp(score), axis = 1) # N * C = 4 * 5
p0 = p0.t() # N * C
#print(p0, '\n')
p0[b, y] += -1 #+ (torch.exp(score[b, y]) / torch.sum(torch.exp(score), axis = 1))
print('p0 =', p0, '\n', 'p =', p)

# refer to: https://github.com/jariasf/CS231n/blob/master/assignment1/cs231n/classifiers/softmax.py
#sum_exp_scores = np.exp(scores).sum(axis=1, keepdims=True)
#softmax_matrix = np.exp(scores)/sum_exp_scores


loss_vec = torch.sum(-torch.log(p))
loss_vec /= num_train
loss_vec += torch.sum(W * W) * reg

dW_vec = X.t().mm(p0)
#p0X = X.t().mm(p0)
#print('p0X =', p0X, '\n', 'W =', W)
#dW_vec_row = torch.sum(p0X, axis=0)
#print("dW_vec_row =", dW_vec_row)
#
#dW_vec = torch.unsqueeze(dW_vec_row, 1).expand(-1, W.shape[1]).clone() # refer to https://stackoverflow.com/questions/59905234/runtimeerror-with-copy-and-expand-unsupported-operation-more-than-one-el
print('dW_vec =', dW_vec)
#dW_vec = dW_vec + 2 * reg * W
dW_vec /= num_train
dW_vec += 2 * reg * W


print('*******************************************************************')
print('score =', score, '\n')
#print('p0=', p0, '\n')
print('dW = ', dW_vec, '\n')
print('p =', p)
#print('l=', l, '\n')
print("loss =", loss_vec)

p0 = tensor([[ 0.1855,  0.1392, -0.8005,  0.2208,  0.2550],
        [-0.8182,  0.1862,  0.1997,  0.1856,  0.2467],
        [ 0.1508, -0.7903,  0.1867,  0.1737,  0.2791],
        [ 0.1716,  0.1886,  0.1828, -0.7476,  0.2046]]) 
 p = tensor([0.1995, 0.1818, 0.2097, 0.2524])
dW_vec = tensor([[ 0.1204, -0.4315, -0.0361, -0.3241,  0.6714],
        [-0.1710, -0.4788, -0.3500,  0.3535,  0.6464],
        [ 0.2030,  0.0923, -0.4731, -0.3617,  0.5396]])
*******************************************************************
score = tensor([[-0.3181, -0.6057, -0.2456, -0.1440,  0.0000],
        [-1.8836, -1.8595, -1.7898, -1.8629, -1.5782],
        [-1.0042, -0.6747, -0.7911, -0.8629, -0.3890],
        [-0.9411, -0.8466, -0.8778, -0.5552, -0.7651]]) 

dW =  tensor([[ 0.0418, -0.0821,  0.0060, -0.0639,  0.1864],
        [-0.0209, -0.1012, -0.0636,  0.1066,  0.1912],
        [ 0.0839,  0.0444, -0.0874, -0.0534,  0.1638]]) 

p = tensor([0.1995, 0.1818, 0.2097, 0.2524])
loss = tensor(1.7887)


In [49]:
# element-wise softmax loss
loss_element = 0
dW_element = torch.zeros_like(W)
#num_train = X.shape[0] # num batch
#num_classes = W.shape[1]
for i in range(num_train):
    score_element = W.t().mv(X[i]) # score of one batch: C * 1 = 5 * 1
    score_element -= torch.max(score_element)
    numerator = torch.exp(score_element[y[i]])
    denominator = torch.sum(torch.exp(score_element))
    p0_element = torch.exp(score_element) / torch.sum(torch.exp(score_element))
    print(y[i])
    p_element = torch.exp(score_element[y[i]]) / torch.sum(torch.exp(score_element))
    print('p =', p_element)
    #print('p0 =', p0_element)
    loss_element += -torch.log(p_element)
    for j in range(num_classes):
        if (j == y[i]):
#            dW_element[:, y[i]] += X[i, :] * (-1 + numerator / denominator)
            dW_element[:, j] += X[i, :] * (-1)
        dW_element[:, j] += X[i, :] * torch.exp(score_element[j]) / denominator
loss_element /= num_train    
loss_element += torch.sum(W * W) * reg
dW_element /= num_train
dW_element += 2 * reg * W

print('score =', score_element, '\n')
#print('p0=', p0, '\n')
print('dW = ', dW_element, '\n')
print('p =', p_element)
#print('l=', l, '\n')
print("loss =", loss_element)

tensor(2)
p = tensor(0.1995)
tensor(0)
p = tensor(0.1818)
tensor(1)
p = tensor(0.2097)
tensor(3)
p = tensor(0.2524)
score = tensor([-0.3859, -0.2914, -0.3226,  0.0000, -0.2099]) 

dW =  tensor([[ 0.0418, -0.0821,  0.0060, -0.0639,  0.1864],
        [-0.0209, -0.1012, -0.0636,  0.1066,  0.1912],
        [ 0.0839,  0.0444, -0.0874, -0.0534,  0.1638]]) 

p = tensor(0.2524)
loss = tensor(1.7887)


In [48]:
# element-wise softmax loss
loss_element = 0
dW_element = torch.zeros_like(W)
#num_train = X.shape[0] # num batch
#num_classes = W.shape[1]
for i in range(num_train):
    score_element = W.t().mv(X[i]) # score of one batch: C * 1 = 5 * 1
    score_element -= torch.max(score_element)
    numerator = torch.exp(score_element[y[i]])
    denominator = torch.sum(torch.exp(score_element))
    p0_element = torch.exp(score_element) / torch.sum(torch.exp(score_element))
    print(y[i])
    p_element = torch.exp(score_element[y[i]]) / torch.sum(torch.exp(score_element))
    print('p =', p_element)
    #print('p0 =', p0_element)
    loss_element += -torch.log(p_element)
    for j in range(num_classes):
        
        dW_element[:, j] += X[i, :] * torch.exp(score_element[j]) / denominator
    dW_element[:, y[i]] -= X[i, :]
loss_element /= num_train    
loss_element += torch.sum(W * W) * reg
dW_element /= num_train
dW_element += 2 * reg * W

print('score =', score_element, '\n')
#print('p0=', p0, '\n')
print('dW = ', dW_element, '\n')
print('p =', p_element)
#print('l=', l, '\n')
print("loss =", loss_element)

tensor(2)
p = tensor(0.1995)
tensor(0)
p = tensor(0.1818)
tensor(1)
p = tensor(0.2097)
tensor(3)
p = tensor(0.2524)
score = tensor([-0.3859, -0.2914, -0.3226,  0.0000, -0.2099]) 

dW =  tensor([[ 0.0418, -0.0821,  0.0060, -0.0639,  0.1864],
        [-0.0209, -0.1012, -0.0636,  0.1066,  0.1912],
        [ 0.0839,  0.0444, -0.0874, -0.0534,  0.1638]]) 

p = tensor(0.2524)
loss = tensor(1.7887)
